## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hanninga (`np.hanning`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:

        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate

lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

In [ ]:
def plot_3_images(image1, image2, image3, suptitle='', title1='Oryginał', title2='Amplituda', title3='Faza'):
    
    plt.figure(figsize=(12,4))
    
    ax = plt.subplot(1, 3, 1)
    ax.imshow(image1, 'gray')
    plt.xticks([]), plt.yticks([])
    plt.title(label=title1)

    ax = plt.subplot(1, 3, 2)
    ax.imshow(image2, 'gray')
    plt.xticks([]), plt.yticks([])
    plt.title(label=title2)

    ax = plt.subplot(1, 3, 3)
    ax.imshow(image3, 'gray')
    plt.xticks([]), plt.yticks([])
    plt.title(label=title3)

    plt.suptitle(t=suptitle)
    plt.show()

def calculate_fourier_dft(image):
    image = image.astype('float32')
    f_image = cv2.dft(image, flags=cv2.DFT_COMPLEX_OUTPUT)
    f_image = np.fft.fftshift(f_image, [0, 1])
    mag, phase = cv2.cartToPolar(f_image[:,:,0], f_image[:,:,1])
    return f_image, mag, phase

def calculate_fourier_dft_log(image):
    image = image.astype('float32')
    f_image = cv2.dft(image, flags=cv2.DFT_COMPLEX_OUTPUT)
    f_image = np.fft.fftshift(f_image, [0, 1])
    mag, phase = cv2.cartToPolar(f_image[:,:,0], f_image[:,:,1])
    return f_image, np.log10(mag + 1), phase

def calculate_fourier_fft(image):
    f_row = np.fft.fft(image, axis=0)
    f_col = np.fft.fft(f_row, axis=1)
    mag, phase = cv2.cartToPolar(f_col.real, f_col.imag)
    return f_col, np.log10(mag + 1), phase

def inverse_fourier(f_image):
    f_image_ishifted = np.fft.ifftshift(f_image, [0, 1])
    inverse_fft = cv2.idft(f_image_ishifted, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    image_ifft = np.round(cv2.magnitude(inverse_fft[:,:,0], inverse_fft[:,:,1])).astype('uint8')
    return image_ifft

In [ ]:
f_lena, _, _ = calculate_fourier_dft_log(lena)

size1d = 21
hanning_window = np.hanning(size1d)
hanning2d = np.outer(hanning_window, hanning_window)

FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(size1d))
FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, size1d]))
FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(size1d))
FSpaceColsM = np.outer(np.ones([1, size1d]), FSpaceCols)

FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

FilterF = FreqR >= 0.2

FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
FilterFidft = cv2.idft(np.float32(FilterFRot3),
                flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

proper_filter = hanning2d*FilterFI

proper_filter_border = cv2.copyMakeBorder(proper_filter, 246, 245, 246, 245, cv2.BORDER_CONSTANT,value=0)

proper_filter_border = proper_filter_border.astype("float32")
filter_dft = cv2.dft(proper_filter_border, flags=cv2.DFT_COMPLEX_OUTPUT)
filter_dft_shifted = np.fft.fftshift(filter_dft, [0,1])

result_filter = np.abs(filter_dft_shifted)

lenaSize=lena.shape
FSpaceRows_512 = 2*np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
FSpaceRowsM_512 = np.outer(FSpaceRows_512, np.ones([1, lenaSize[1]]))
FSpaceCols_512 = 2*np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
FSpaceColsM_512 = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols_512)  


figFilter = plt.figure()
axsFilter = figFilter.add_subplot(projection='3d')
axsFilter.plot_surface(FSpaceRowsM_512, FSpaceColsM_512, result_filter[:,:,0], rstride=1, cstride=1, 
                       cmap=plt.get_cmap('gray'), linewidth=0)



f_lena_filtered = f_lena*result_filter

i_lena = inverse_fourier(f_lena_filtered)

plot_3_images(lena, i_lena, np.abs(lena-i_lena), suptitle='Poprawiony filtr bez artefaktów', 
              title1='Oryginał', title2='Po odwrotnej transformacie', title3='Różnica bezwzględna')